In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [3]:
# 전처리된 train, test set 부르기
train = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/preprocessed_dataset/ppc_train.csv")
test = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/preprocessed_dataset/ppc_test.csv")

In [4]:
train.head()

,Unnamed: 0,index,title,topic_idx,cleaned_title
0,0,0,인천에서핀란드 항공기 결항 휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통
1,1,1,실리콘밸리 넘어서겠다 구글 조원 들여 미국전역 거점화,4,실리콘밸리 넘어서겠다 구글 조원 들여 전역 거점화
2,2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3,3,3,클린턴 측근한국기업 특수관계 조명 공과 사 맞물려,4,클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합
4,4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망


# 1. vectorizing (tf-idf)

In [4]:
train_text = train["cleaned_title"].tolist()
test_text = test["cleaned_title"].tolist()
train_label = np.asarray(train.topic_idx)

tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')

In [11]:
train_tf_text # 각 행은 문장(기사 제목)이고, 각 열은 문장에서 나오는 모든 단어들, 각 셀은 그 단어들의 tf-idf 값

<45654x150000 sparse matrix of type '<class 'numpy.float32'>'
	with 393497 stored elements in Compressed Sparse Row format>

# 2. modeling

https://ebbnflow.tistory.com/120

In [5]:
def dnn_model():
  model = Sequential() # 신경망의 뼈대(?) 느낌. 여기에 우리가 .add()를 추가하여 각 레이어에 원하는 옵션을 추가해주면 됨
  model.add(Dense(128, input_dim = 150000, activation = "relu")) 
  # Dense()
  # 첫번째 인자 : output node의 수
  # 두번째 인자 (input_dim) : 맨 처음 레이어의 input node의 수 (즉 train의 크기)
  # 세번째 인자 : activation function의 설정 
  model.add(Dropout(0.8)) # keep_prob(unit이 유지될 확률) = 0.8인 dropout 기법 사용
  model.add(Dense(7, activation = "softmax")) # 마지막 레이어에 classification을 위한 softmax 활성화함수 사용
  return model

In [6]:
model = dnn_model()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.optimizers.Adam(0.001), metrics = ['accuracy'])

In [20]:
history = model.fit(x = train_tf_text[:40000], y = train_label[:40000],
                    validation_data =(train_tf_text[40000:], train_label[40000:]),
                    epochs = 6)

Epoch 1/6
1250/1250 [==============================] - 17s 14ms/step - loss: 0.2565 - accuracy: 0.9275 - val_loss: 0.6424 - val_accuracy: 0.7885
Epoch 2/6
1250/1250 [==============================] - 17s 14ms/step - loss: 0.2050 - accuracy: 0.9433 - val_loss: 0.6462 - val_accuracy: 0.7872
Epoch 3/6
1250/1250 [==============================] - 17s 14ms/step - loss: 0.1595 - accuracy: 0.9552 - val_loss: 0.6518 - val_accuracy: 0.7876
Epoch 4/6
1250/1250 [==============================] - 17s 13ms/step - loss: 0.1355 - accuracy: 0.9625 - val_loss: 0.6672 - val_accuracy: 0.7871
Epoch 5/6
1250/1250 [==============================] - 17s 13ms/step - loss: 0.1168 - accuracy: 0.9684 - val_loss: 0.6772 - val_accuracy: 0.7846
Epoch 6/6
1250/1250 [==============================] - 17s 13ms/step - loss: 0.0976 - accuracy: 0.9730 - val_loss: 0.6956 - val_accuracy: 0.7812


# 3. predict

In [21]:
# training error
tr_pred = model.predict(train_tf_text)
pred = np.argmax(tr_pred, axis = 1)

1427/1427 [==============================] - 3s 2ms/step


In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(train.topic_idx, pred)

0.9660708809742848

In [8]:
# predict test set
tmp_pred = model.predict(test_tf_text)
pred = np.argmax(tmp_pred, axis = 1)

286/286 [==============================] - 1s 4ms/step
